# ノイズ除去

このノートブックでは、ノイズ除去について学びます。

## 目次
1. [ガウシアンノイズ](#ガウシアンノイズ)
2. [ソルト&ペッパーノイズ](#ソルト&ペッパーノイズ)
3. [メディアンフィルタ](#メディアンフィルタ)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from skimage import filters, feature

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["axes.unicode_minus"] = False
np.random.seed(42)
print("ライブラリのインポートが完了しました。")

## ガウシアンノイズ

In [ ]:
# ガウシアンノイズの除去
# ガウシアンノイズは、正規分布に従うランダムノイズです

# 元の画像
img = np.zeros((200, 200), dtype=np.uint8)
img[50:150, 50:150] = 200  # 明るい領域

# ガウシアンノイズの追加
mean = 0
std = 25
noise = np.random.normal(mean, std, img.shape)
noisy_img = np.clip(img.astype(np.float32) + noise, 0, 255).astype(np.uint8)

# ガウシアンフィルタでノイズ除去
denoised_gaussian = cv2.GaussianBlur(noisy_img, (5, 5), 0)

# 異なるカーネルサイズで比較
denoised_3x3 = cv2.GaussianBlur(noisy_img, (3, 3), 0)
denoised_7x7 = cv2.GaussianBlur(noisy_img, (7, 7), 0)
denoised_9x9 = cv2.GaussianBlur(noisy_img, (9, 9), 0)

# 結果の表示
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('元の画像')
axes[0, 0].axis('off')

axes[0, 1].imshow(noisy_img, cmap='gray')
axes[0, 1].set_title('ガウシアンノイズ付き')
axes[0, 1].axis('off')

axes[0, 2].imshow(denoised_3x3, cmap='gray')
axes[0, 2].set_title('ガウシアンフィルタ 3x3')
axes[0, 2].axis('off')

axes[1, 0].imshow(denoised_gaussian, cmap='gray')
axes[1, 0].set_title('ガウシアンフィルタ 5x5')
axes[1, 0].axis('off')

axes[1, 1].imshow(denoised_7x7, cmap='gray')
axes[1, 1].set_title('ガウシアンフィルタ 7x7')
axes[1, 1].axis('off')

axes[1, 2].imshow(denoised_9x9, cmap='gray')
axes[1, 2].set_title('ガウシアンフィルタ 9x9')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("ガウシアンノイズの除去:")
print("- ガウシアンフィルタが最も効果的")
print("- カーネルサイズが大きいほど強く平滑化（細部が失われる）")
print("- バランスの取れたサイズ（5x5）が推奨")

## ソルト&ペッパーノイズ

In [ ]:
# ソルト&ペッパーノイズの除去
# ソルト&ペッパーノイズは、ランダムに白点（ソルト）と黒点（ペッパー）が現れるノイズです

# 元の画像
img = np.zeros((200, 200), dtype=np.uint8)
img[50:150, 50:150] = 200

# ソルト&ペッパーノイズの追加
salt_pepper_img = img.copy()
# ランダムに白点と黒点を追加
salt_mask = np.random.random((200, 200)) < 0.05
pepper_mask = np.random.random((200, 200)) < 0.05
salt_pepper_img[salt_mask] = 255
salt_pepper_img[pepper_mask] = 0

# メディアンフィルタでノイズ除去
denoised_median = cv2.medianBlur(salt_pepper_img, 5)

# 異なるカーネルサイズで比較
denoised_median_3 = cv2.medianBlur(salt_pepper_img, 3)
denoised_median_7 = cv2.medianBlur(salt_pepper_img, 7)

# ガウシアンフィルタとの比較
denoised_gaussian = cv2.GaussianBlur(salt_pepper_img, (5, 5), 0)

# 結果の表示
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('元の画像')
axes[0, 0].axis('off')

axes[0, 1].imshow(salt_pepper_img, cmap='gray')
axes[0, 1].set_title('ソルト&ペッパーノイズ')
axes[0, 1].axis('off')

axes[0, 2].imshow(denoised_median_3, cmap='gray')
axes[0, 2].set_title('メディアンフィルタ 3x3')
axes[0, 2].axis('off')

axes[1, 0].imshow(denoised_median, cmap='gray')
axes[1, 0].set_title('メディアンフィルタ 5x5（推奨）')
axes[1, 0].axis('off')

axes[1, 1].imshow(denoised_median_7, cmap='gray')
axes[1, 1].set_title('メディアンフィルタ 7x7')
axes[1, 1].axis('off')

axes[1, 2].imshow(denoised_gaussian, cmap='gray')
axes[1, 2].set_title('ガウシアンフィルタ（比較）')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("ソルト&ペッパーノイズの除去:")
print("- メディアンフィルタが最も効果的")
print("- ガウシアンフィルタは効果が限定的")
print("- メディアンフィルタは外れ値に強い")
print("- エッジを保持しながらノイズを除去")

## メディアンフィルタ

In [ ]:
# メディアンフィルタの詳細
# メディアンフィルタは、ソルト&ペッパーノイズに特に有効です

# 複合ノイズの除去
img_complex = img.copy()
# ガウシアンノイズ
gaussian_noise = np.random.normal(0, 15, img.shape)
img_complex = np.clip(img_complex.astype(np.float32) + gaussian_noise, 0, 255).astype(np.uint8)
# ソルト&ペッパーノイズ
salt_mask = np.random.random((200, 200)) < 0.03
pepper_mask = np.random.random((200, 200)) < 0.03
img_complex[salt_mask] = 255
img_complex[pepper_mask] = 0

# 様々なフィルタでノイズ除去
denoised_median = cv2.medianBlur(img_complex, 5)
denoised_gaussian = cv2.GaussianBlur(img_complex, (5, 5), 0)
denoised_bilateral = cv2.bilateralFilter(img_complex, 9, 75, 75)

# 組み合わせ: メディアン → ガウシアン
denoised_combined = cv2.GaussianBlur(denoised_median, (3, 3), 0)

# 結果の表示
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('元の画像')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_complex, cmap='gray')
axes[0, 1].set_title('複合ノイズ')
axes[0, 1].axis('off')

axes[0, 2].imshow(denoised_median, cmap='gray')
axes[0, 2].set_title('メディアンフィルタ')
axes[0, 2].axis('off')

axes[1, 0].imshow(denoised_gaussian, cmap='gray')
axes[1, 0].set_title('ガウシアンフィルタ')
axes[1, 0].axis('off')

axes[1, 1].imshow(denoised_bilateral, cmap='gray')
axes[1, 1].set_title('バイラテラルフィルタ')
axes[1, 1].axis('off')

axes[1, 2].imshow(denoised_combined, cmap='gray')
axes[1, 2].set_title('メディアン→ガウシアン')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("\n=== 各フィルタの特徴 ===")
print("メディアンフィルタ:")
print("  - ソルト&ペッパーノイズに最適")
print("  - エッジを保持")
print("  - 外れ値に強い")
print("\nガウシアンフィルタ:")
print("  - ガウシアンノイズに最適")
print("  - 自然な平滑化")
print("  - エッジが少しぼやける")
print("\nバイラテラルフィルタ:")
print("  - エッジを保持しながら平滑化")
print("  - 計算コストが高い")
print("  - 色情報を考慮")
print("\n組み合わせ:")
print("  - 複数のフィルタを順次適用")
print("  - 異なる種類のノイズに対応")